<CENTER>
    <a href="http://opendata.atlas.cern/release/2020/documentation/notebooks/intro.html" class="icons"><img src="../../images/ATLASOD.gif" style="width:40%"></a>
</CENTER>

<CENTER><h1>Searching for the Higgs boson in the H&#8594;&gamma;&gamma; channel (using C++) </h1></CENTER>

## C++ notebook example

**Introduction**
Let's take a current ATLAS Open Data sample and create a histogram:

In order to activate the interactive visualisation of the histogram that is later created we can use the JSROOT magic:

In [1]:
%jsroot on

We need to include some standard C++ and ROOT libraries

In [2]:
// Creates a TChain to be used by the Analysis.C class
#include <TChain.h>
#include <vector>
#include <TFile.h>
#include <iostream>
#include <string>
#include <stdio.h>

Because we would like to use more than one ROOT input file, the best option is to use a TChain object. This allows to "chain" several samples into a single structure that we can later loop over

In [3]:
TString path = "https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/GamGam/"

(TString &) "https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/GamGam/"[70]


In [4]:
TChain* fChain = new TChain("mini");

fChain->AddFile(path+"Data/data_A.GamGam.root");
fChain->AddFile(path+"Data/data_B.GamGam.root");
fChain->AddFile(path+"Data/data_C.GamGam.root");
fChain->AddFile(path+"Data/data_D.GamGam.root");

Now we're going to extract the photons variables

In [5]:
UInt_t  Photon_n = -1;  //number of preselected photons

vector<float>   *Photon_pt;  //transverse momentum of the photon
vector<float>   *Photon_eta = 0;  //pseudorapidity of the photon
vector<float>   *Photon_phi = 0;  //azimuthal angle of the photon
vector<float>   *Photon_E   = 0;  //energy of the photon
vector<bool>    *Photon_isTightID = 0;

Bool_t          TrigP = 0;
vector<float>   *Photon_ptcone30 = 0;
vector<float>   *Photon_etcone20 = 0;

Here we're filling the variables defined above with the content of those inside the input ntuples

In [6]:
fChain->SetBranchAddress("photon_pt",        &Photon_pt);  
fChain->SetBranchAddress("photon_n",         &Photon_n);  

fChain->SetBranchAddress("photon_eta",       &Photon_eta);
fChain->SetBranchAddress("photon_phi",       &Photon_phi);
fChain->SetBranchAddress("photon_E",         &Photon_E);
fChain->SetBranchAddress("photon_isTightID", &Photon_isTightID);

fChain->SetBranchAddress("trigP",            &TrigP);
fChain->SetBranchAddress("photon_ptcone30",  &Photon_ptcone30);
fChain->SetBranchAddress("photon_etcone20",  &Photon_etcone20);

We're creating a histogram for this example. The plan in to fill them with events.

In [7]:
//Invariant mass histograms definition
TH1F *h_M_Hyy = new TH1F("h_M_Hyy","Diphoton invariant-mass ; M_{#gamma#gamma} [GeV] ; Events / bin", 30, 105, 160);

We are selecting below a simple look for them.

In [8]:
h_M_Hyy->SetMarkerSize(2.0);
h_M_Hyy->SetLineColor(kBlue);
h_M_Hyy->SetFillColor(kBlue-10);

The Higgs boson analysis implemented here considers Higgs boson decays into a photon pair. The event selection criteria are (this will take a few minutes):

In [ ]:
int nentries, nbytes, i;
nentries = (Int_t)fChain->GetEntries();

// Message at the beginning to indicate how many entries are to be analyzed
std::cout << "* Total number of entries to analyse: " << nentries << std::endl;

// Preselection of good photons
for (i = 0; i < nentries; i++)
{
    nbytes = fChain->GetEntry(i);

    // Cut: Diphoton trigger is satisfied
    if (TrigP)
    {
        int goodphoton_index[5];  // give the vector a size good enough to avoid segmentation faults due to events with more than 2 photons.
        int goodphoton_n = 0;     // the total number of good photons
        int photon_index = 0;

        // Loop over all photons and apply cuts
        for (unsigned int j = 0; j < Photon_n; j++)
        {
            // Cut: photons are tight
            if (Photon_isTightID->at(j))
            {
                // Cut: excluding the transition region between the barrel and endcap calorimeters
                if (TMath::Abs(Photon_eta->at(j)) < 2.37 &&
                    (TMath::Abs(Photon_eta->at(j)) < 1.37 || TMath::Abs(Photon_eta->at(j)) > 1.52))
                {
                    // Collect the photon that passes the cut
                    goodphoton_index[photon_index] = j;
                    goodphoton_n++;
                    photon_index++;
                }
            }
        }

        // Cut: Exactly two photons that passed the eta and isTightID cuts
        if (goodphoton_n == 2)
        {
            int goodphoton1_index = goodphoton_index[0];
            int goodphoton2_index = goodphoton_index[1];

            // Cut: pT cut - photon 1 has pT > 40 GeV and photon 2 has pT > 30 GeV
            if (Photon_pt->at(goodphoton1_index) > 40000. && Photon_pt->at(goodphoton2_index) > 30000.)
            {
                // Cut: isolation cut - using absolute value of ptcone30 (in MeV)
                if (Photon_ptcone30->at(goodphoton1_index) < 4000. && Photon_ptcone30->at(goodphoton2_index) < 4000.)
                {
                    // TLorentzVector definitions
                    TLorentzVector Photon_1 = TLorentzVector();
                    TLorentzVector Photon_2 = TLorentzVector();

                    Photon_1.SetPtEtaPhiE(Photon_pt->at(goodphoton1_index), Photon_eta->at(goodphoton1_index), Photon_phi->at(goodphoton1_index), Photon_E->at(goodphoton1_index));
                    Photon_2.SetPtEtaPhiE(Photon_pt->at(goodphoton2_index), Photon_eta->at(goodphoton2_index), Photon_phi->at(goodphoton2_index), Photon_E->at(goodphoton2_index));

                    // Calculation of the Invariant Mass using TLorentz vectors
                    TLorentzVector Photon_12 = Photon_1 + Photon_2;
                    float mass_inv_GeV = Photon_12.M() / 1000.;

                    h_M_Hyy->Fill(mass_inv_GeV);
                }
            }
        }
    } // end TrigPhoton request
}

// Final message after analysis
std::cout << "* Analysed a total of: " << nentries << " in this sample." << std::endl;


#### Final plot

In [ ]:
TCanvas *cz = new TCanvas("cz", "cz", 900, 600);

// Set up the main pad for the fit plot
TPad *mainPad = new TPad("mainPad", "mainPad", 0, 0.3, 1, 1.0);
mainPad->SetBottomMargin(0);  // Upper and lower pads share the X axis
mainPad->Draw();
mainPad->cd();

h_M_Hyy->SetMarkerStyle(20);  // Data markers
h_M_Hyy->SetMarkerSize(0.8);
h_M_Hyy->SetLineColor(kBlack);
h_M_Hyy->SetStats(0);  // Hide stats box

// Define the signal + background fit (Gaussian for signal + 4th order polynomial for background)
TF1 *sigPlusBkg = new TF1("sigPlusBkg", "gaus(0) + pol4(3)", 105, 160);  // Gaussian + 4th order polynomial
sigPlusBkg->SetParameters(1000, 125, 2);  // Parameters for the Gaussian (amplitude, mean, sigma)
sigPlusBkg->SetLineColor(kRed);
sigPlusBkg->SetLineStyle(1);

// Define the background-only fit (4th-order polynomial)
TF1 *bkgOnly = new TF1("bkgOnly", "pol4", 105, 160);  // 4th order polynomial for background
bkgOnly->SetLineColor(kRed);
bkgOnly->SetLineStyle(2);  // Dashed line for background

// Perform the fit (quiet mode to suppress fit output)
h_M_Hyy->Fit("sigPlusBkg", "RQ");  // Signal + background fit
h_M_Hyy->Fit("bkgOnly", "RQ+");    // Background-only fit

// Draw the data points with error bars
h_M_Hyy->Draw("E1");
sigPlusBkg->Draw("same");  // Draw signal + background fit
bkgOnly->Draw("same");     // Draw background-only fit

// Add legend to main plot
TLegend *legend = new TLegend(0.55, 0.65, 0.9, 0.85);
legend->AddEntry(h_M_Hyy, "Data", "lep");
legend->AddEntry(sigPlusBkg, "Sig+Bkg Fit (m_H = 125 GeV)", "l");
legend->AddEntry(bkgOnly, "Bkg (4th order polynomial)", "l");
legend->Draw();

// Add text labels (ATLAS Open Data, Luminosity, and channel info)
TLatex latex;
latex.SetTextSize(0.04);
latex.DrawLatexNDC(0.2, 0.85, "ATLAS Open Data");
latex.DrawLatexNDC(0.2, 0.8, "#sqrt{s}=13 TeV, #intLdt = 10 fb^{-1}");
latex.DrawLatexNDC(0.2, 0.75, "H #rightarrow #gamma#gamma");

cz->cd();

// Create the residual pad
TPad *residPad = new TPad("residPad", "residPad", 0, 0.05, 1, 0.3);
residPad->SetTopMargin(0);
residPad->SetBottomMargin(0.3);
residPad->Draw();
residPad->cd();

// Calculate residuals (Data - Background)
int nBins = h_M_Hyy->GetNbinsX();
TH1F *residuals = new TH1F("residuals", "", nBins, 105, 160);  // Residuals (data - background)

// Loop over all bins to calculate residuals and plot them
for (int i = 1; i <= nBins; i++) {
    double dataValue = h_M_Hyy->GetBinContent(i);
    double bkgValue = bkgOnly->Eval(h_M_Hyy->GetBinCenter(i));  // Background value at bin center
    double error = h_M_Hyy->GetBinError(i);

    // Residuals = (Data - Background) / Error
    double residualValue = (dataValue - bkgValue) / error;
    
    residuals->SetBinContent(i, residualValue);  // Set residual (Data - Bkg)
}

// Style residual plot components
residuals->SetMarkerStyle(20);  // Data points
residuals->SetMarkerSize(0.8);
residuals->SetLineColor(kBlack);

// Customize axis labels and scales
residuals->GetYaxis()->SetTitle("Events - Bkg");
residuals->GetYaxis()->SetTitleSize(0.07);
residuals->GetYaxis()->SetTitleOffset(0.5);
residuals->GetYaxis()->SetLabelSize(0.07);
residuals->GetXaxis()->SetTitle("di-photon invariant mass m_{#gamma#gamma} [GeV]");
residuals->GetXaxis()->SetTitleSize(0.07);
residuals->GetXaxis()->SetLabelSize(0.07);

// Draw the residuals (Data - Background)
residuals->Draw("E1");

// Create a Gaussian with mean 125, sigma 2 (same as your fit) for the residual plot
TF1 *gaussianResidual = new TF1("gaussianResidual", "gaus(0)", 105, 160);
gaussianResidual->SetParameters(sigPlusBkg->GetParameter(0), sigPlusBkg->GetParameter(1), sigPlusBkg->GetParameter(2));  // Gaussian params: amplitude, mean, sigma
gaussianResidual->SetLineColor(kRed);  // Red color for Gaussian
gaussianResidual->SetLineStyle(1);     // Solid line for Gaussian
gaussianResidual->Draw("same");        // Draw the Gaussian signal on the residual plot

// Draw the dashed red line at Y=0 for the background minus background
TLine *zeroLine = new TLine(105, 0, 160, 0);  // Line from (x=105, y=0) to (x=160, y=0)
zeroLine->SetLineColor(kRed);
zeroLine->SetLineStyle(2);  // Dashed line
zeroLine->Draw("same");

// Display the canvas
cz->Draw();


#### Log Scale

In [ ]:
h_M_Hyy->Draw("E3");
cz->SetLogy();
cz->Draw();

**Done!**